This notebook takes the image/mask pairs sorted by extract_dcm_for_wsx, and:
 - loads them, gets the areas of fat shown in the manually-created masks
 - maps them back to the pairs created by matching_diabetics_healthy_controls
 - writes out a sensibly-formatted spreadsheet, with one healthy/diabetic pair per row
 - does some basic statistics allowing us to draw conclusions about whether or not we are measuring a sensible thing

In [ ]:
import pandas as pd

import numpy as np